In [ ]:
import os
import sys
import folium.features

# Obtenir le répertoire de travail courant
current_dir = os.getcwd()

# Obtenir le répertoire parent de `maps` (qui est `src`)
parent_dir = os.path.abspath(os.path.join(current_dir, os.pardir))

# Ajouter `src` au chemin de recherche des modules
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

In [ ]:
import matplotlib.pyplot as plt

from city_utils import *
from useful_methods.data_processing import extract_data


In [ ]:
df = pd.read_csv("../../database/data.csv", sep=";", decimal=',')
df_extracted = extract_data(df, provider='Orange', techno='4g')

In [ ]:
labels_1 = city_detection(df_extracted[['x', 'y']])['labels']
labels_2 = city_detection_enhanced(df_extracted[['x', 'y']])['labels']

In [ ]:
def labelsToColor(l1, l2): 
    if ((l1==-1) and (l2==-1)):
        return 'green'
    if((l1==-1) or (l2==-1)):
        return 'blue'
    return 'red'

colors = pd.Series(data=[labelsToColor(l1, l2) for l1, l2 in zip(labels_1, labels_2)], index=df_extracted.index)
map = folium.Map(location=[46.8, 2.3], zoom_start=7, tiles="Cartodb Positron")

for ind, latitude, longitude in df_extracted[['latitude', 'longitude']].itertuples():
    # Convertir les coordonnées Lambert 93 en longitude/latitude
    popup = f'ville (DBScan, HDBScan) :\n({labels_1[ind]!=-1},{labels_2[ind]!=-1})'
    map.add_child(folium.RegularPolygonMarker(location=[latitude, longitude], fill_opacity=0.2, radius=1.5, color = colors[ind], popup=popup))

map.save("../../out/maps/city-detection-comparison.html")

In [ ]:
city_comparison(labels_1, labels_2)